In [1]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.get(name="quick-starts-ws-127732")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127732
Azure region: southcentralus
Subscription id: 3929502f-a035-4029-a2f5-865565e57a0a
Resource group: aml-quickstarts-127732


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
from azureml.core.compute_target import ComputeTargetException

compute_name = "udacity-cluster"
try:
    compute = ComputeTarget(workspace=ws, name=compute_name)
    print('Compute cluster {} already exists!'.format(compute_name))
except ComputeTargetException:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute = ComputeTarget.create(ws, compute_name, config)
    
compute.wait_for_completion()

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
# Commenting out this line since I decided to use another sampling strategy
# from azureml.train.hyperdrive.parameter_expressions import uniform 
import os

# Specify parameter sampler
from azureml.train.hyperdrive import loguniform, choice
ps = RandomParameterSampling(
    {
        '--C': loguniform(-5, 3), 
        '--max_iter': choice(50, 100, 500, 1000, 5000, 10000)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=100, slack_factor=0.1)

# Create a SKLearn estimator for use with train.py
from azureml.train.estimator import Estimator
est = Estimator(source_directory=".",
                entry_script="train.py",
                compute_target=compute,
                conda_packages=['scikit-learn==0.21.3', 'pandas==0.23.4']
               )


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=100)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_metrics())
print(best_run.get_file_names())
os.makedirs('./models', exist_ok=True)
best_run.download_file(name='outputs/model.pkl', output_file_path='./models/lr_bankmarketing.pkl')
best_run.register_model(model_name='lr_bankmarketing', model_path='outputs/model.pkl')

{'Regularization Strength:': 0.024412623517399486, 'Max iterations:': 50, 'Accuracy': 0.9191198786039454}
['azureml-logs/55_azureml-execution-tvmps_3804b7271d2b1adf5ad54977eeb73406c8b8f1a52707e610fd33ef36209e4c32_d.txt', 'azureml-logs/65_job_prep-tvmps_3804b7271d2b1adf5ad54977eeb73406c8b8f1a52707e610fd33ef36209e4c32_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_3804b7271d2b1adf5ad54977eeb73406c8b8f1a52707e610fd33ef36209e4c32_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/106_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_389845b4-773d-442a-9631-2e305f2b3d74.jsonl', 'logs/azureml/dataprep/python_span_l_389845b4-773d-442a-9631-2e305f2b3d74.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.pkl']


Model(workspace=Workspace.create(name='quick-starts-ws-127732', subscription_id='3929502f-a035-4029-a2f5-865565e57a0a', resource_group='aml-quickstarts-127732'), name=lr_bankmarketing, id=lr_bankmarketing:1, version=1, tags={}, properties={})

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(url)

In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
from train import clean_data
x, y = clean_data(dataset)
td = x.join(y)
os.makedirs('./data', exist_ok=True)
td.to_csv('data/bankmarketing_train_cleaned.csv')
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='data', overwrite=True, show_progress=True)

dataset = TabularDatasetFactory.from_delimited_files(path=ds.path('data/bankmarketing_train_cleaned.csv'))

Uploading an estimated of 1 files
Uploading ./data/bankmarketing_train_cleaned.csv
Uploaded ./data/bankmarketing_train_cleaned.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    compute_target=compute,
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name="y",
    n_cross_validations=5)

In [9]:
# Submit your automl run
automl_run = exp.submit(automl_config, show_output=True)

Running on remote.
Running on remote compute: udacity-cluster
Parent Run ID: AutoML_ab8da5eb-e67b-4ac7-ba2d-25304bf2d174

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+--------------------------------

In [10]:
# Retrieve and save your best automl model.
best_automl_run, best_automl_model = automl_run.get_output()
print(best_automl_run.get_metrics())
joblib.dump(value=best_automl_model, filename="./models/automl_bankmarketing.pkl")
from azureml.core.model import Model
Model.register(model_path = "./models/automl_bankmarketing.pkl",
                       model_name = "automl_bankmarketing",
                       description = "Bank marketing model trained by AutoML",
                       workspace = ws)

{'balanced_accuracy': 0.7513349661315081, 'recall_score_micro': 0.9168740515933234, 'average_precision_score_macro': 0.8266719429057398, 'precision_score_micro': 0.9168740515933234, 'accuracy': 0.9168740515933234, 'precision_score_macro': 0.8006794303937508, 'weighted_accuracy': 0.9580025714792482, 'AUC_macro': 0.9484123099720906, 'f1_score_micro': 0.9168740515933231, 'recall_score_weighted': 0.9168740515933234, 'norm_macro_recall': 0.5026699322630162, 'average_precision_score_micro': 0.981782144062407, 'average_precision_score_weighted': 0.9559800427143236, 'precision_score_weighted': 0.9111314201803935, 'AUC_weighted': 0.9484123099720906, 'recall_score_macro': 0.7513349661315081, 'log_loss': 0.1850640378675149, 'f1_score_weighted': 0.9131872966625105, 'f1_score_macro': 0.7728195230777833, 'matthews_correlation': 0.5497331088093207, 'AUC_micro': 0.9809878626971938, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_ab8da5eb-e67b-4ac7-ba2d-25304bf2d174_24/confusion_matrix'

Model(workspace=Workspace.create(name='quick-starts-ws-127732', subscription_id='3929502f-a035-4029-a2f5-865565e57a0a', resource_group='aml-quickstarts-127732'), name=automl_bankmarketing, id=automl_bankmarketing:1, version=1, tags={}, properties={})

In [11]:
run.log('Hyperdrive logistic regression regularization (C)', best_run.get_metrics()['Regularization Strength:'])
run.log('Hyperdrive logistic regression max_iter', best_run.get_metrics()['Max iterations:']) 
run.log('Hyperdrive logistic regression accuracy', best_run.get_metrics()['Accuracy'])
run.log('AutoML model', best_automl_model.steps[-1][0])
run.log('AutoML accuracy', best_automl_run.get_metrics()['accuracy'])

In [12]:
# Clean up compute cluster
try:
    compute.delete()
except ComputeTargetException as e:
    print(e.message)
    print("Failed to clean up compute cluster!")
compute.wait_for_completion(show_output=True, is_delete_operation=True)

Deleting......Current provisioning state of AmlCompute is "Deleting"

...
SucceededProvisioning operation finished, operation "Succeeded"


In [13]:
try:
    ComputeTarget(workspace=ws, name=compute_name)
except ComputeTargetException:
    print('Compute cluster {} no longer exists.'.format(compute_name))

Compute cluster udacity-cluster no longer exists.
